# Hypersonic TPS Simulator #

In [ ]:
import numpy as np
import json
import physics 
from solvers import *

In [ ]:
input = (8, 60, 60, 2)

def driver(mach, altitude, t_max, material_id):
    """
    Driver function to run the hypersonic TPS simulation.

    Parameters:
        mach: Free-stream Mach number
        altitude: Altitude in meters
        t_max: Maximum simulation time in seconds
        material_id: Material ID for TPS material

    Returns:
        results: Dictionary containing time, temperature profile, and TPS thickness over time
    """
    

    # Load parameters from JSON file
    with open('config/params.json', 'r') as f:
        params = json.load(f)

    # Update parameters with input values
    params['M_inf'] = mach
    params['altitude'] = altitude
    params['t_max'] = t_max
    params['material_id'] = str(material_id)